In [1]:
import sys
sys.path.append('/mnt/lovit/git/soynlp/')
import soynlp

몇 개의 문장에 대해서 학습된 모델 기반으로 작동하는 형태소 분석기를 이용하여 명사를 추출하여 봅시다.

이 튜토리얼을 만들 때의 KoNLPy의 버전은 0.4.4입니다. 

In [2]:
import konlpy
from konlpy.tag import Kkma, Twitter, Hannanum

kkma = Kkma()
twitter = Twitter()
hannanum = Hannanum()
print('konlpy version = %s' % konlpy.__version__)

konlpy version = 0.4.4


'박근혜'라는 단어는 여러 정치 관련 문서에서 등장하였기 때문에 사전에 등장할 가능성이 높음에도, 꼬꼬마의 경우 박근혜를 '박근 + 혜'와 '박근혜' 중 어떤 것이 명사인지 혼동합니다. '최순실'의 경우 학습데이터에 이 단어가 들어갈리가 거의 없기 때문에 대부분 형태소분석기들이 '최순실'이라는 명사를 인식하지 못합니다. 

In [3]:
sent = '박근혜 최순실의 국정농단 사태는 대통령탄핵으로 이어졌습니다'
print('꼬꼬마 명사: ', kkma.nouns(sent))
print('트위터 명사: ', twitter.nouns(sent))
print('한나눔 명사: ', hannanum.nouns(sent))

꼬꼬마 명사:  ['박근', '박근혜', '혜', '최', '최순실의', '순', '실의', '국정', '국정농단', '농단', '사태', '대통령', '대통령탄핵', '탄핵']
트위터 명사:  ['박근혜', '최순', '실의', '국정', '농단', '사태', '대통령', '탄핵']
한나눔 명사:  ['박근혜', '최순실', '국정농단', '사태', '대통령탄핵', '이']


위와 비슷하게 '트와이스', '아이오아이'이 시간이 지나면서 발생하는 단어들은 명사로 인식하기가 매우 힘듭니다. 모르는 어절이 입력되면 이를 명사로 판단하는 휴리스틱들을 대부분 이용합니다. 사실 많은 양의 단어들은 명사이기 때문에 이는 매우 합리적인 fallback입니다. 트와이스 노래 'tt'의 경우에는 심지어 외국어이기 때문에 애초에 외국어로 분리가 되어 명사로 인식되지 못합니다. 

In [4]:
sent = '트와이스 그리고 아이오아이 좋아여 tt가 저번에 1위 했었죠?'
print('꼬꼬마 명사: ', kkma.nouns(sent))
print('트위터 명사: ', twitter.nouns(sent))
print('한나눔 명사: ', hannanum.nouns(sent))

꼬꼬마 명사:  ['트와', '트와이스', '이스', '아이오', '아이오아이', '아이', '여', '저번', '1', '1위', '위']
트위터 명사:  ['트와이스', '아이오', '아이', '저번', '위']
한나눔 명사:  ['트와이스', '아이오아', '좋아', '저번', '1위']


'보코하람'과 같이 외국어가 들어오면 이를 분해하는 특징도 있습니다. '보코하람'은 단어로 알지 못하지만 '보', '코' 라는 것은 명사로 알고 있기 때문입니다. 

In [5]:
sent = '보코하람 테러로 소말리아에서 전쟁이 있었어요'
print('꼬꼬마 명사: ', kkma.nouns(sent))
print('트위터 명사: ', twitter.nouns(sent))
print('한나눔 명사: ', hannanum.nouns(sent))

꼬꼬마 명사:  ['보', '보코', '코', '테러', '소말리', '전쟁']
트위터 명사:  ['보코하람', '테러', '소말리아', '전쟁']
한나눔 명사:  ['보코하람', '테러', '소말리아', '전쟁']


하지만 이는 학습데이터의 관점에서 인식하기 어려운 명사이지, 매일 뉴스를 보는 우리는 모두 인식할 수 있는 단어들입니다. 

    오늘 A에서 저녁먹자
    너랑간 A 진짜 좋더라
    기다릴테니까 곧바로 A로 와
    난 A가 좋더라
    
위 세 문장을 보면 A라는 단어는 명사라는 것을 우리는 쉽게 눈치챕니다. 왜냐면 A 다음에 오는 단어들이 [-에서, -로, -가]와 같이 명사 다음에 오는 조사들이 자주 등장하기 때문입니다. 

한국어는 L + [R] 구조입니다. (이 내용은 Word_extraction_tutorial.ipynb를 보고 오시기 바랍니다.) L 옆에 등장하는 R의 분포는 L이 명사인지 아닌지를 판단하는 좋은 힌트가 됩니다. 하지만 조사들을 R로 가지고 있으면서 규칙기반으로 명사를 찾을 수는 없습니다. '-은'은 대표적인 조사입니다. 하지만 '김정은'이 '김정 + 은'은 아닙니다. 북한 관련 뉴스들을 자주 보면 '김정은'은 자주 등장하지만 '김정 + 은', '김정 + 이', '김정 + 에게' 같은 어절은 자주 등장하지 않습니다. 

우리는 이와 같이 L - R의 bipartite graph의 정보를 이용하여 해당 단어가 명사인지 아닌지 판단합니다. 세종 말뭉치를 이용하여 명사 뒤에 등장하는 R set을 모아뒀으며, R set의 단어들은 명사 가능 점수가 학습되어 있습니다. 명사 가능 점수의 범위는 [-1, 1]입니다. 

    내서	   -0.530702
    있게	    1.000000
    있는	    0.327824
    쓰는	    0.079298
    었다며	   -1.000000
    였다며	    0.437399
    
'했 + 었다며'이라면 '했'은 명사 점수가 -1.0입니다. '재미 + 있게' 3번, '재미 + 있는' 2번 등장하였다면 재미의 명사 가능 점수는 (3 * 1.0 + 2 * 0.33) / 5 = 0.732점입니다. 명사 추출의 threshold가 0.5라면 '재미'는 명사로 추출됩니다. 

이 방법은 주어진 문서집합에서 어절들의 구조를 학습하여 그 주어진 문서집합의 명사를 추출합니다. 학습데이터가 필요하지 않은 통계 기반의 unsupervised 학습방법입니다. 

In [2]:
from soynlp.noun import LRNounExtractor
from soynlp.word import WordExtractor

verbose=True
left_max_length=10
right_max_length=6
min_count=5

word_extractor = WordExtractor()
noun_extractor = LRNounExtractor(None, 
                                 verbose, 
                                 left_max_length, 
                                 right_max_length, 
                                 min_count, 
                                 word_extractor)

used default noun predictor; Sejong corpus predictor
used noun_predictor_sejong
2398 r features was loaded


NounExtractor.train(sents)의 sents는 len(sents)를 할 수 있는 list-like이면 됩니다. 

    __iter__
    __len__ 
    
두 함수만 구현되어 있으면 됩니다. 튜토리얼에 이용한 문서는 2016년 10월 20일의 뉴스로부터 한글로 이뤄진 223,357 문장입니다. 

In [4]:
class Sentences:
    def __init__(self, fname):
        self.fname = fname
        self.length = 0
    def __iter__(self):
        with open(self.fname, encoding='utf-8') as f:
            for doc in f:
                doc = doc.strip()
                if not doc:
                    continue
                for sent in doc.split('  '):
                    yield sent
    def __len__(self):
        if self.length == 0:
            with open(self.fname, encoding='utf-8') as f:
                for doc in f:
                    doc = doc.strip()
                    if not doc:
                        continue
                    self.length += len(doc.split('  '))
        return self.length
    
corpus_fname = 'MY_DoubleSpaceLine_Corpus'
sentences = Sentences(corpus_fname)
len(sentences)

223357

추출하고 싶은 명사의 noun score threshold와 명사의 최소빈도수 (min count)를 parameter로 넣으면 명사 점수 0.3점 이상, 문서집합에서 등장한 빈도가 100번 이상인 단어들에서 명사를 추출합니다. 

train 함수에서는 WordExtractor를 통하여 단어를 미리 추출하고, 이들 중에서 명사를 판단합니다. 22만개 문장으로부터 단어를 추출하고 명사를 판단하는 학습시간은 i7-5820 기준 39초가 걸렸습니다. 

In [5]:
%%time
nouns = noun_extractor.train_extract(sentences, minimum_noun_score=0.3, min_count=100)
print('num nouns = %d' % len(nouns))

scanning completed
(L,R) has (160842, 79334) tokens
training was done. used memory 0.352 Gb
all cohesion probabilities was computed. # words = 223348
all branching entropies was computed # words = 95906
all accessor variety was computed # words = 95906
num nouns = 10900
CPU times: user 39.6 s, sys: 204 ms, total: 39.8 s
Wall time: 39.6 s


다음의 단어들이 명사로 추출되었는지 살펴보겠습니다. 

In [6]:
words = ['박근혜', '우병우', '민정수석', '트와이스', '아이오아이']
for word in words:
    print('%s is noun? %r' % (word, word in nouns))

박근혜 is noun? True
우병우 is noun? True
민정수석 is noun? True
트와이스 is noun? True
아이오아이 is noun? True


is_noun을 이용하면 각각의 단어에 대하여 명사 여부를 판단할 수도 있습니다. 

In [7]:
noun_extractor.is_noun('최순실', minimum_noun_score=0.3)

True

몇 개의 단어들을 입력하여 명사점수를 추출할 수도 있습니다. 

In [8]:
words = ['박근혜', '우병우', '민정수석', '트와이스', '아이오아이', '최순실', '최순실게이트', '게이트', '콘서트']
nouns_ = noun_extractor.extract(words, minimum_noun_score=0.3, min_count=100)
for word, score in nouns_.items():
    print('%s: %.3f' % (word, score[0]))

민정수석: 0.889
우병우: 0.784
트와이스: 0.544
콘서트: 0.846
아이오아이: 0.716
최순실: 0.875
박근혜: 0.632
게이트: 0.831


명사 점수 0.3점 이상이면서 빈도수가 100 이 넘는 단어들을 (빈도수, 빈도수 * 명사점수) 두 가지 기준으로 상위 100개씩 선택하여 프린트를 하겠습니다. 

In [9]:
def k_words_a_line(top, k=5):
    def as_str(subtop):
        return '    '.join(['%6s (%.2f)' % (t[0], t[1][0]) for t in subtop])
    num_line = len(top) // k if len(top) % k == 0 else (len(top) // k) + 1
    sents = []
    for i in range(num_line):
        sents.append(as_str(top[i*k:(i+1)*k]))
    return '\n'.join(sents)

def frequency(word):
    return noun_extractor.word_extractor.frequency(word)[0]

In [10]:
top100 = sorted(nouns.items(), key=lambda x:frequency(x[0]), reverse=True)[:100]
print(k_words_a_line(top100))

    20 (0.31)        기자 (0.75)        서울 (0.51)        무단 (1.00)       20일 (0.94)
   재배포 (1.00)        금지 (0.99)        지난 (0.99)        사진 (0.55)      2016 (0.86)
   것으로 (0.43)       뉴시스 (1.00)        한국 (0.85)        전재 (0.40)        진행 (0.95)
    함께 (0.98)        이번 (1.00)        오후 (0.99)      저작권자 (1.00)        우리 (0.70)
    미국 (0.85)        관련 (0.98)        이날 (0.76)       뉴스1 (0.96)       대통령 (0.67)
    참석 (0.98)       19일 (0.93)       이라고 (0.59)        오전 (1.00)        사업 (0.65)
    프로 (0.67)        북한 (0.86)        중국 (0.80)        이상 (0.89)        세계 (0.99)
   때문에 (0.43)        문제 (0.80)        최근 (0.98)        영화 (0.71)      연합뉴스 (0.81)
    국내 (0.93)        투자 (0.87)        배우 (0.53)        운영 (0.92)        올해 (0.61)
    가장 (0.97)        이후 (0.98)        라고 (0.61)        현재 (0.78)        생각 (0.85)
    교육 (0.70)        문화 (0.92)        조사 (0.96)        확인 (0.95)       트럼프 (0.55)
    기대 (0.95)        부산 (0.64)        필요 (0.98)       스포츠 (0.78)        모두 (0.99)
    한편 (0.98)   

In [11]:
top100 = sorted(nouns.items(), key=lambda x:x[1][0] * frequency(x[0]), reverse=True)[:100]
print(k_words_a_line(top100))

    기자 (0.75)        무단 (1.00)       재배포 (1.00)        금지 (0.99)       20일 (0.94)
    20 (0.31)        지난 (0.99)        서울 (0.51)      2016 (0.86)       뉴시스 (1.00)
    한국 (0.85)        함께 (0.98)        이번 (1.00)        진행 (0.95)        오후 (0.99)
  저작권자 (1.00)        사진 (0.55)        관련 (0.98)       뉴스1 (0.96)        참석 (0.98)
    미국 (0.85)        오전 (1.00)        우리 (0.70)       19일 (0.93)        이날 (0.76)
    세계 (0.99)        최근 (0.98)       것으로 (0.43)        이상 (0.89)        북한 (0.86)
    이후 (0.98)        국내 (0.93)        가장 (0.97)        운영 (0.92)       대통령 (0.67)
    투자 (0.87)        중국 (0.80)      연합뉴스 (0.81)        문제 (0.80)        조사 (0.96)
    문화 (0.92)        확인 (0.95)        생각 (0.85)        필요 (0.98)        모두 (0.99)
    프로 (0.67)        영화 (0.71)        현재 (0.78)        한편 (0.98)        기대 (0.95)
    사업 (0.65)        전재 (0.40)       이라고 (0.59)        출연 (0.96)        참여 (0.96)
   11월 (0.98)        당시 (0.91)        이용 (0.87)      있습니다 (0.95)        결과 (0.83)
    이런 (1.00)   